In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import LabelEncoder

In [12]:
# Hyper-parameters 
input_size = 14
hidden_size = 10
num_classes = 2
num_epochs = 5
batch_size = 100
learning_rate = 0.01

In [13]:
# Read in data
data = pd.read_csv("adult.data",header = None)

# Drop patients' ID column
categorical_features = [1,3,5,6,7,8,9,13,14]

# Encode categorical variables
label_encoders = {}
for i in categorical_features:
    label_encoders[i] = LabelEncoder()
    data[i] = label_encoders[i].fit_transform(data[i])


In [14]:
class AdultData(Dataset):
    """Customized class that pre-processes adult dataset"""
    def __init__(self, csv_file):
        """
        Args:
            data: The audlt pandas dataframe 
        """
        self.main = data
        self.n = len(data)

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        features = self.main.iloc[idx, 0:14].values
        features = features.astype('double')
 
        features = torch.from_numpy(features)
        
            
        label = self.main.iloc[idx,14]

        sample = (features,label)


        return sample

In [15]:
# Get the dataset through the class
dataset = AdultData(data)

In [16]:
# Specify portion for splitting, shuffle data, set seed
percent_split = .2
shuffle_dataset = True
random_seed= 5

# Creating data indices for training and testing splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(percent_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [17]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #CUDA is a GPU


In [18]:
#Neural Network Class 
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        torch.manual_seed(10)
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

In [19]:
# Set model
model = NeuralNet(input_size, hidden_size, num_classes).double().to(device)

In [20]:
# Set loss function and optimize algorithm
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Set start time
start = time.time()

for epoch in range(num_epochs):
    train_loss, test_loss = [], []
    correct = 0
    total = 0
    # Train the model
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 57).to(device)
        images = images.to(device)
        labels = labels.to(device=device, dtype=torch.int64)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    
    with torch.no_grad():
        
        # Predict the model
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device=device, dtype=torch.int64)
            outputs = model(images)
            lose = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss.append(lose.item())
            
        print ("Epoch:", epoch + 1, ", Training Loss: ", np.mean(train_loss), ", Test loss: ", np.mean(test_loss))

# Set end time         
end = time.time()  
print('Accuracy is: {} %'.format(100 * correct / total))
print('It takes ' +  str(end - start)  + ' seconds to run Feedforward Neural Network algorithm on adult dataset.')

Epoch: 1 , Training Loss:  26.78654466009395 , Test loss:  0.5452199154967271
Epoch: 2 , Training Loss:  0.5440319064800538 , Test loss:  0.5420638506647799
Epoch: 3 , Training Loss:  0.5354074150155947 , Test loss:  0.5464692832313354
Epoch: 4 , Training Loss:  0.5538830887644124 , Test loss:  0.5378702732631314
Epoch: 5 , Training Loss:  0.5199367500886004 , Test loss:  0.5020331209203956
Accuracy is: 79.6068796068796 %
It takes 65.54642081260681 seconds to run Feedforward Neural Network algorithm on adult dataset.
